# Toolbox di Valutazione per l'Interpolazione di Frame con TimeLens

## Introduzione
Questo notebook implementa il calcolo delle metriche MSE, PSNR, SSIM e LPIPS per la valutazione di frame interpolati. I dati devono essere organizzati in una struttura specifica:

### Struttura delle Cartelle
    
    └── results_folder
            ├── sequence_0
            │   ├── GT               <------ Dove sono salvate le immagini di ground truth
            │   │   ├── 000000.png
            │   │   ├── 000001.png
            │   │   └── ...
            │   ├── method_0          <------ Il primo metodo da valutare
            │   │   ├── 000000.png
            │   │   ├── 000001.png
            │   │   └── ...
            │   ├── method_1
            │   │   ├── 000000.png
            │   │   ├── 000001.png
            │   │   └── ...
            │   └── ...
            ├── sequence_1
            │   ├── GT
            │   │   ├── 000000.png
            │   │   ├── 000001.png
            │   │   └── ...
            │   └── ...
            └── ...
            
## Requisiti
- Python 3.x
- Librerie: `numpy`, `cv2`, `lpips`

In [1]:
# Importa librerie necessarie
import os
from os.path import join, basename
import glob
import cv2
import numpy as np
import lpips

# Inizializza LPIPS
loss_fn = lpips.LPIPS(net='vgg')  # Usa 'alex' o 'vgg'

/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <FB2FD416-6C4D-3621-B677-61F07C02A3C5> /opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/homebrew/Caskroom/miniforge/base/envs/timelens/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan 

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/marcello/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:12<00:00, 45.3MB/s] 


Loading model from: /opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/lpips/weights/v0.1/vgg.pth


/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.

In [23]:
from evaluation import compute_mse, compute_psnr, compute_ssim, compute_lpips


def process_dataset(gt_files, pred_files, num_skips, grayscale):
    metrics = {"MSE": [], "PSNR": [], "SSIM": [], "LPIPS": []}

    for i, (gt_path, pred_path) in enumerate(zip(gt_files, pred_files)):
        if i % (num_skips + 1) != 0:
            continue

        gt_img = cv2.imread(gt_path)
        pred_img = cv2.imread(pred_path)

        if gt_img is None or pred_img is None or gt_img.shape != pred_img.shape:
            print(f"Errore con immagini: GT={gt_path}, Pred={pred_path}")
            continue

        if grayscale:
            gt_img = cv2.cvtColor(gt_img, cv2.COLOR_BGR2GRAY)
            pred_img = cv2.cvtColor(pred_img, cv2.COLOR_BGR2GRAY)

        try:
            metrics["MSE"].append(compute_mse(gt_img, pred_img))
            metrics["PSNR"].append(compute_psnr(gt_img, pred_img))
            metrics["SSIM"].append(compute_ssim(gt_img, pred_img))
            metrics["LPIPS"].append(compute_lpips(gt_img, pred_img))
        except Exception as e:
            print(f"Errore nel calcolo metriche: {e}")

    return metrics

In [24]:
def evaluate(parent_folder, filter_string="test", num_skips=0, grayscale=False):
    """
    Valuta tutti i dataset all'interno delle cartelle che includono la stringa `filter_string`.

    Args:
        parent_folder (str): Cartella principale contenente le cartelle come ev_test, ev_test2, ecc.
        filter_string (str): Stringa da usare per filtrare le directory principali.
        num_skips (int): Numero di frame da saltare per la valutazione.
        grayscale (bool): Se True, converte le immagini in scala di grigi prima della valutazione.
    """
    # Trova tutte le directory principali che contengono il filtro
    main_folders = [d for d in sorted(glob.glob(join(parent_folder, f"*{filter_string}*"))) if os.path.isdir(d)]

    if not main_folders:
        print(f"Nessuna cartella trovata con il filtro '{filter_string}' in {parent_folder}")
        return

    # Itera attraverso ogni cartella principale (es. ev_test, ev_test2, ...)
    for main_folder in main_folders:
        print(f"\n--- Analisi in {main_folder} ---")

        # Trova tutte le sottocartelle dei dataset (es. baloon_popping, spinning_plate, ecc.)
        dataset_folders = [d for d in sorted(glob.glob(join(main_folder, "*"))) if os.path.isdir(d)]

        if not dataset_folders:
            print(f"Nessun dataset trovato in {main_folder}, skipping.")
            continue

        # Processa ogni dataset
        for dataset_path in dataset_folders:
            dataset_name = basename(dataset_path)
            print(f"\n--- Dataset: {dataset_name} ---")

            # Trova le sottocartelle (GT, Timelens, ecc.)
            subfolders = [f for f in sorted(glob.glob(join(dataset_path, "*"))) if os.path.isdir(f)]
            method_files = {basename(sf): sorted(glob.glob(join(sf, "*.png"))) for sf in subfolders}

            # Controlla se esiste la cartella GT
            if "GT" not in method_files:
                print(f"Manca cartella GT in {dataset_name}, skipping.")
                continue

            # Estrai i file di GT
            gt_files = method_files.pop("GT")

            # Processa ogni metodo (es. Timelens, AltroMetodo, ecc.)
            for method_name, pred_files in method_files.items():
                if len(gt_files) != len(pred_files):
                    print(f"Mismatch file GT({len(gt_files)}) e {method_name}({len(pred_files)})")
                    continue

                # Calcola le metriche
                metrics = process_dataset(gt_files, pred_files, num_skips, grayscale)
                print(f"Metodo: {method_name}")
                for metric, values in metrics.items():
                    mean = np.mean(values)
                    std = np.std(values)
                    print(f"  {metric}: {mean:.4f} ± {std:.4f}")

In [25]:
# Parametri
test_folder = "test_folder"
num_skips = 1  # Numero di frame da saltare
grayscale = False  # Calcolo in scala di grigi

evaluate(test_folder, num_skips=num_skips, grayscale=grayscale)


--- Analisi in test_folder/ev_test ---

--- Dataset: baloon_popping ---
Metodo: Timelens
  MSE: 19.3768 ± 1.1346
  PSNR: 33.7382 ± 1.2608
  SSIM: 0.7603 ± 0.0449
  LPIPS: nan ± nan

--- Analisi in test_folder/ev_test2 ---

--- Dataset: fountaine ---


/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/opt/homebrew/Caskroom/miniforge/base/envs/timelens/lib/python3.9/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divi

Metodo: Timelens
  MSE: 162.3000 ± 29.2467
  PSNR: 25.9236 ± 0.6333
  SSIM: 0.6640 ± 0.0230
  LPIPS: nan ± nan

--- Dataset: spinning_plate ---


KeyboardInterrupt: 